# Fluvial Hazard Zone Prediction: Problem Statement

<img style="-webkit-user-select: none;background-position: 0px 0px, 10px 10px;background-size: 20px 20px;background-image:linear-gradient(45deg, #eee 25%, transparent 25%, transparent 75%, #eee 75%, #eee 100%),linear-gradient(45deg, #eee 25%, white 25%, white 75%, #eee 75%, #eee 100%);cursor: zoom-in;" src="https://www.denverpost.com/wp-content/uploads/2016/04/20131102__helifloodp1.jpg?w=600" width="439" height="292">

In 2013, the Colorado Front Range experienced historic flooding caused by an 8 day rainfall event.  Rainfall totals were approximated as a 1,000 year event.  Flooding forced evacuations of thousands of residents, hundreds of home were destroyed and 4 lives were lost. 

http://mediacenter.dailycamera.com/2013/09/12/photos-massive-flash-flooding-along-front-range-of-colorado/#29

Floods of this scale produce the natural phenomenon of channel migration - or the movement of rivers from one location to another within their valleys.  Although flood maps produced by the Federal Emergency Management Agency (FEMA) predict the extent of high water during the 100 year flood, they do not capture the potential channel migration hazards associated with erosion, deposition, degradation, lateral migration, and avulsion.

A number of state and federal agencies are investigating methodologies to identify these channel migration zones using geomorphic, geologic, hydraulic, and physical data along with a healthy dose of expert knowledge. Colorado has recently released a guidance document for use in these delineation efforts.  

http://coloradohazardmapping.com/hazardMapping/fluvialMapping

However, these delineation efforts are time intensive and difficult to scale.  What is currently lacking is a method to extend this knowlege to a landscape scale to encompass the large number of river miles where people live in harm's way.

Using pre-flood and post-flood topography, this project aims to add a new tool to the prediction of channel migration zones (CMZs).  We'll compare various machine learning algorithms' and artificial neural networks' abilities to predict Channel Migration Zones along river corridors due to the 2013 floods. 


#  The South St. Vrain River.  Lyons, Colorado.  
This reach of river experienced significant channel migration
which damaged roads and infrastructure.  In addition to having detailed pre and 
post flood topography here, the Colorado Water Conservation Board is conducting a 
Fluvial Hazard Zone Delineation of this reach using an expert-based geomorphic analysis.  Their effort
will provide a comparison to the results we generate in this project.

In [1]:
import pandas as pd
from mapboxgl.utils import *
from mapboxgl.viz import *
import pysal.esda.mapclassify as mapclassify

# The Data
We've gridded the problem area into 3ft by 3ft cells within a 500 foot buffer of the South Saint Vrain. Within that grid, we've queried the source datasets for information on topographic characteristics, infrastructure, spatial relationships, and the change in ground elevation after the 2013 floods.  Latitude and longitude also uniquely identify each 3ft by 3ft cell. 

In [25]:
csv = r'/Users/Daniel/Documents/Programming/Project_Scripts/CMZ/data/SouthSaintVrain_points.csv'
df = pd.read_csv(csv, header=0)
df.tail(3)

,FID,2011topo,stream_slope,near_crossing,near_road,ground_delta,near_stream,ground_curve,relative_elevation,ground_slope,long_WGS84,lat_WGS84
906127,906127,5492.770020,1.42103,1786.630005,334.078003,-0.199219,603,2.51465,-9999.0,22.253401,-105.284137,40.207955
906128,906128,5491.200195,1.42103,1784.160034,334.007996,-0.037109,603,2.12809,-9999.0,22.329500,-105.284127,40.207955
906129,906129,5490.479981,1.42103,1781.680054,333.937012,-0.619629,604,1.69497,-9999.0,21.482700,-105.284116,40.207955


In [31]:
# From https://github.com/bokeh/datashader/blob/master/datashader/utils.py
import numpy as np
# Convert WGS toWeb Mercator coordinates (meters East of Greenwich and meters North of the Equator).


def lnglat_to_meters(longitude, latitude):
    """
    Projects the given (longitude, latitude) values into Web Mercator
    coordinates (meters East of Greenwich and meters North of the Equator).
    Longitude and latitude can be provided as scalars, Pandas columns,
    or Numpy arrays, and will be returned in the same form.  Lists
    or tuples will be converted to Numpy arrays.
    Examples:
       easting, northing = lnglat_to_meters(-40.71,74)
       easting, northing = lnglat_to_meters(np.array([-74]),np.array([40.71]))
       df=pandas.DataFrame(dict(longitude=np.array([-74]),latitude=np.array([40.71])))
       df.loc[:, 'longitude'], df.loc[:, 'latitude'] = lnglat_to_meters(df.longitude,df.latitude)
    """
    if isinstance(longitude, (list, tuple)):
        longitude = np.array(longitude)
    if isinstance(latitude, (list, tuple)):
        latitude = np.array(latitude)

    origin_shift = np.pi * 6378137
    easting = longitude * origin_shift / 180.0
    northing = np.log(np.tan((90 + latitude) * np.pi / 360.0)) * origin_shift / np.pi
    return (easting, northing)

df.loc[:, 'longitude'], df.loc[:, 'latitude'] = lnglat_to_meters(df.long_WGS84,df.lat_WGS84)

print (df.tail())

           FID     2011topo  stream_slope  near_crossing   near_road  \
906125  906125  5495.660156       1.42103    1791.599976  334.218994   
906126  906126  5494.529785       1.42103    1789.109985  334.148987   
906127  906127  5492.770020       1.42103    1786.630005  334.078003   
906128  906128  5491.200195       1.42103    1784.160034  334.007996   
906129  906129  5490.479981       1.42103    1781.680054  333.937012   

        ground_delta  near_stream  ground_curve  relative_elevation  \
906125     -0.437988          602       3.67839             -9999.0   
906126     -0.587891          602       4.01566             -9999.0   
906127     -0.199219          603       2.51465             -9999.0   
906128     -0.037109          603       2.12809             -9999.0   
906129     -0.619629          604       1.69497             -9999.0   

        ground_slope  long_WGS84  lat_WGS84     longitude      latitude  
906125     19.959101 -105.284159  40.207955 -1.172018e+07  4.89620

In [21]:
import datashader.transfer_functions as tf
import datashader as ds
# import pandas as pd

# background='black'
from datashader.bokeh_ext import InteractiveImage

In [26]:
import bokeh.plotting as bp
from bokeh.models.tiles import WMTSTileSource
bp.output_notebook()

def base_plot(tools='pan,wheel_zoom,reset',webgl=False):
    p = bp.figure(tools=tools,
        plot_width=int(850), plot_height=int(500),
        x_range=x_range, y_range=y_range, outline_line_color=None,
        min_border=0, min_border_left=0, min_border_right=0,
        min_border_top=0, min_border_bottom=0) #, webgl=webgl)

    p.axis.visible = False
    p.xgrid.grid_line_color = None
    p.ygrid.grid_line_color = None

    return p

Loading BokehJS ...

In [32]:
background = "black"
from datashader.bokeh_ext import InteractiveImage

def image_callback(x_range, y_range, w, h, color_fn=tf.shade):
    cvs = ds.Canvas(plot_width=w, plot_height=h, x_range=x_range, y_range=y_range)
    agg = cvs.points(df, 'longitude', 'latitude', ds.count('relative_elevation'))
    image = color_fn(agg)
    return tf.dynspread(image,threshold=0.75, max_px=8)
    #return(image)

x_range, y_range = (-8242000, -8210000), (4965000,4990000)
    
p = base_plot()
show(p)

url="https://server.arcgisonline.com/ArcGIS/rest/services/World_Imagery/MapServer/tile/{Z}/{Y}/{X}.png"
# #url="http://tile.stamen.com/toner-background/{Z}/{X}/{Y}.png"
tile_renderer = p.add_tile(WMTSTileSource(url=url))
tile_renderer.alpha=1.0 if background == "black" else 0.15

InteractiveImage(p, image_callback)

W-1001 (NO_DATA_RENDERERS): Plot has no data renderers: Figure(id='ec5503db-0f7a-4654-bb5c-edc0561a5524', ...)


W-1001 (NO_DATA_RENDERERS): Plot has no data renderers: Figure(id='e43065d5-b7ad-40d0-b921-a33dfeae9d71', ...)
W-1001 (NO_DATA_RENDERERS): Plot has no data renderers: Figure(id='ec5503db-0f7a-4654-bb5c-edc0561a5524', ...)


In [9]:
from bokeh.io import output_file, show
from bokeh.models import GeoJSONDataSource
from bokeh.plotting import figure
from bokeh.sampledata.sample_geojson import geojson

# geo_source = GeoJSONDataSource(geojson='points_sample.geojson')
geojson = 'points_sample.geojson'
geo_source = GeoJSONDataSource(geojson=geojson)

p = figure()
p.circle(x='x', y='y', alpha=0.9, source=geo_source)
output_file("geojson.html")
show(p)

ValueError: expected JSON text, got 'points_sample.geojson'

In [3]:
# Must be a public token, starting with `pk`
token = os.getenv('MAPBOX_ACCESS_TOKEN','pk.eyJ1IjoiaW5kaWVseXQiLCJhIjoiY2pkcXZyMGZpMDB6NzJxbGw4aXdvb2w3bCJ9.sL_EzvrSj83Y0Hi1_6GT6A')

In [4]:
# Create a downsampled version of the full dataframe for plotting.
df_sample = df.sample(frac=0.25, replace=False)

# Create a geojson file export from the current dataframe
df_to_geojson(df_sample, filename='points_sample.geojson',
              properties=['2011topo', 'ground_delta', 'relative_elevation'], 
                     lat='lat_WGS84', lon='long_WGS84', precision=8)

{'feature_count': 226532, 'filename': 'points_sample.geojson', 'type': 'file'}

In [5]:
# Generate data breaks and color stops from colorBrewer
color_breaks = mapclassify.Natural_Breaks(df['relative_elevation'], k=10, initial=0).bins
color_stops = create_color_stops(color_breaks, colors='RdYlGn')

# Create the viz from the sampled dataframe
if True:
    viz = CircleViz('points_sample.geojson',
                    access_token=token, 
                    height='400px',
                    color_property = "relative_elevation",
                    color_stops = color_stops,
                    center = (-105.279,40.2136),
                    zoom = 12,
                    below_layer = 'waterway-label',
                    opacity=0.25)

viz.style_url='mapbox://styles/mapbox/satellite-streets-v9'
viz.show()
